In [3]:
!pip install langchain langchain-openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 55.0 MB/s eta 0:00:00


In [5]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [ ]:
os.environ['OPENAI_API_KEY'] = 'sk'


#Prueba con Chatbot sobre consulta tu normas

In [9]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import os

# Configurar la API key de OpenAI
os.environ['OPENAI_API_KEY'] = 'sk'

# Definir el prompt personalizado
custom_template = """
Sistema: Utiliza el siguiente contexto para responder la pregunta del usuario.
Si el artículo específico no se encuentra completo en el contexto, proporciona un resumen detallado
de la información disponible y menciona que es un resumen parcial basado en las referencias encontradas.
Si no encuentras información sobre el artículo solicitado, indícalo claramente.

Contexto:
{context}

Pregunta del usuario: {question}

Instrucciones adicionales:
1. Si encuentras el artículo completo, muéstralo en su totalidad
2. Si solo encuentras referencias parciales, elabora un resumen estructurado
3. Incluye cualquier referencia cruzada relevante a otros artículos
4. Menciona explícitamente si la información proporcionada es parcial o completa

Respuesta:"""

def crear_base_conocimiento(ruta_archivo):
    loader = TextLoader(ruta_archivo)
    documentos = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,  # Aumentado para capturar más contexto
        chunk_overlap=400,  # Aumentado para mejor coherencia
        length_function=len
    )
    chunks = text_splitter.split_documents(documentos)

    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(chunks, embeddings)

    return vectorstore

def crear_chatbot(vectorstore):
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0
    )

    # Configurar el prompt personalizado
    prompt = PromptTemplate(
        template=custom_template,
        input_variables=["context", "question"]
    )

    memory = ConversationBufferMemory(
        memory_key="chat_history",
        output_key="answer",
        return_messages=True
    )

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(
            search_kwargs={"k": 4}  # Aumentar el número de documentos recuperados
        ),
        memory=memory,
        return_source_documents=True,
        combine_docs_chain_kwargs={'prompt': prompt},
        verbose=True
    )

    return chain

def iniciar_chat():
    print("Inicializando chatbot...")

    try:
        vectorstore = crear_base_conocimiento("normas_ue.txt")
    except Exception as e:
        print(f"Error al cargar el archivo: {str(e)}")
        return

    chain = crear_chatbot(vectorstore)

    print("\n¡Bienvenido al Chatbot de Normativas UE!")
    print("Este bot te ayudará a entender las normativas de la UE.")
    print("Escribe 'salir' para terminar.")

    while True:
        pregunta = input("\nTu pregunta: ")

        if pregunta.lower() == 'salir':
            break

        try:
            resultado = chain.invoke({"question": pregunta})

            print("\nRespuesta:")
            print(resultado["answer"])

            # Mostrar las fuentes de manera más estructurada
            if resultado.get("source_documents"):
                print("\nFuentes consultadas:")
                fuentes_unicas = set()
                for doc in resultado["source_documents"]:
                    # Extraer y limpiar el contenido para evitar duplicados
                    contenido = doc.page_content.strip()
                    if contenido not in fuentes_unicas:
                        fuentes_unicas.add(contenido)

                for i, fuente in enumerate(fuentes_unicas, 1):
                    print(f"\nFuente {i}:")
                    print(fuente)
                    print("-" * 80)

        except Exception as e:
            print(f"Error al procesar la pregunta: {str(e)}")

    print("\n¡Gracias por usar el chatbot!")

if __name__ == "__main__":
    iniciar_chat()

Inicializando chatbot...

¡Bienvenido al Chatbot de Normativas UE!
Este bot te ayudará a entender las normativas de la UE.
Escribe 'salir' para terminar.

Tu pregunta: dime de que trata el articulo 10


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Sistema: Utiliza el siguiente contexto para responder la pregunta del usuario.
Si el artículo específico no se encuentra completo en el contexto, proporciona un resumen detallado 
de la información disponible y menciona que es un resumen parcial basado en las referencias encontradas.
Si no encuentras información sobre el artículo solicitado, indícalo claramente.

Contexto:
Artículo 32

Acceso a la justicia

1.   Toda persona física o jurídica que tenga un interés suficiente, tal como se determine de conformidad con las vías nacionales de recurso legales que existan, incluyendo el supuesto de que dichas personas cumplan, de exigirse, los criterios establecidos en el Derecho nacional, 

KeyboardInterrupt: Interrupted by user

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

class DataProcessor:
    def __init__(self):
        self.label_encoders = {}
        self.scaler = StandardScaler()

    def preprocess_company_data(self, df):
        df_processed = df.copy()

        # Procesar características numéricas
        numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
        for feature in numeric_features:
            df_processed[feature] = df_processed[feature].fillna(df_processed[feature].median())

        # Procesar características categóricas
        categorical_features = df.select_dtypes(include=['object']).columns
        for feature in categorical_features:
            if feature not in self.label_encoders:
                self.label_encoders[feature] = LabelEncoder()
                df_processed[feature] = self.label_encoders[feature].fit_transform(
                    df_processed[feature].fillna('MISSING')
                )

        # Características derivadas
        df_processed['risk_ratio'] = df_processed['MONTO_MULTA'] / df_processed['numtra'].replace(0, 1)
        df_processed['worker_density'] = df_processed['numtra'] / (df_processed['costosal'].replace(0, 1))
        df_processed['safety_index'] = df_processed['ntrab_consctr'] / df_processed['numtra'].replace(0, 1)

        return df_processed

    def preprocess_single_company(self, company):
        return self.preprocess_company_data(pd.DataFrame([company]))

class RobustLightGBM:
    def __init__(self):
        self.model = None
        self.feature_names = None

    def train(self, X_train, y_train, X_test, y_test):
        self.feature_names = X_train.columns.tolist()

        print("\nPreparando datos para LightGBM...")
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

        params = {
            'objective': 'multiclass',
            'num_class': 5,
            'metric': ['multi_logloss', 'multi_error'],
            'num_leaves': 31,
            'learning_rate': 0.03,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'min_child_samples': 20,
            'verbose': -1
        }

        print("Entrenando modelo robusto...")
        callbacks = [
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]

        self.model = lgb.train(
            params,
            train_data,
            num_boost_round=1000,
            valid_sets=[valid_data],
            callbacks=callbacks
        )

        y_pred = self.predict(X_test)
        y_test_adjusted = y_test + 1

        accuracy = accuracy_score(y_test_adjusted, y_pred)
        conf_matrix = confusion_matrix(y_test_adjusted, y_pred)

        print(f"\nPrecisión del modelo: {accuracy:.2%}")
        print("\nMatriz de confusión:")
        print(conf_matrix)

        importance = pd.DataFrame({
            'feature': self.feature_names,
            'importance': self.model.feature_importance()
        }).sort_values('importance', ascending=False)

        print("\nCaracterísticas más importantes:")
        print(importance.head())

    def predict(self, X):
        probabilities = self.model.predict(X[self.feature_names])
        return np.argmax(probabilities, axis=1) + 1

    def predict_single(self, X):
        return self.predict(X)[0]

class RobustComplianceEvaluator:
    def __init__(self):
        self.data_processor = DataProcessor()
        self.model = RobustLightGBM()
        self.company_data = None
        self.processed_data = None

    def load_data(self, file_path):
        try:
            self.company_data = pd.read_csv(file_path)
            self.processed_data = self.data_processor.preprocess_company_data(self.company_data)
            print(f"Datos cargados exitosamente. Shape: {self.company_data.shape}")
            print(f"Columnas disponibles: {len(self.company_data.columns)}")
            return True
        except Exception as e:
            print(f"Error al cargar datos: {str(e)}")
            return False

    def train_model(self):
        try:
            features = self.get_important_features()
            X = self.processed_data[features]

            compliance_scores = self.processed_data['compliance_score'].copy()
            mean_score = compliance_scores.mean()
            compliance_scores = compliance_scores.fillna(mean_score)

            # Crear categorías de estrellas (0-4 para LightGBM)
            conditions = [
                (compliance_scores <= 65),
                (compliance_scores > 65) & (compliance_scores <= 75),
                (compliance_scores > 75) & (compliance_scores <= 85),
                (compliance_scores > 85) & (compliance_scores <= 95),
                (compliance_scores > 95)
            ]
            values = [0, 1, 2, 3, 4]
            y = pd.Series(np.select(conditions, values, default=2))

            print("\nDistribución de estrellas:")
            distribution = y.value_counts().sort_index()
            distribution.index = distribution.index + 1
            print(distribution)

            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42, stratify=y
            )

            print(f"\nDatos de entrenamiento: {X_train.shape}")
            print(f"Datos de prueba: {X_test.shape}")

            self.model.train(X_train, y_train, X_test, y_test)
            return True

        except Exception as e:
            print(f"Error en el entrenamiento: {str(e)}")
            return False

    def get_important_features(self):
        return [
            'risk_score',
            'MONTO_MULTA',
            'ntrab_tc_indet',
            'ntrab_hombres',
            'ntrab_mujeres',
            'ntrab_consctr',
            'TRAB_AFEC',
            'costosal',
            'Tiempo_E_anio',
            'prop_indefinidos',
            'ntrab_afil_essalud',
            'risk_ratio',
            'worker_density',
            'safety_index'
        ]

    def evaluate_company(self, company_id):
        try:
            if company_id not in self.company_data['ID_EMPRESA'].values:
                return {
                    'error': 'ID de empresa no encontrado',
                    'valid_ids': self.company_data['ID_EMPRESA'].unique().tolist()
                }

            company = self.company_data[self.company_data['ID_EMPRESA'] == company_id].iloc[0]
            processed_company = self.data_processor.preprocess_single_company(company)

            features = self.get_important_features()
            prediction = self.model.predict_single(processed_company[features])

            return self.prepare_detailed_evaluation(company, prediction)

        except Exception as e:
            print(f"Error en la evaluación: {str(e)}")
            return {'error': 'Error en la evaluación'}

    def prepare_detailed_evaluation(self, company, prediction):
        stars = int(prediction)
        risk_level = "Alto" if company['risk_score'] > 7 else "Medio" if company['risk_score'] > 4 else "Bajo"

        if stars >= 4:
            compliance_status = "Excelente"
            color_code = "Verde"
        elif stars == 3:
            compliance_status = "Bueno"
            color_code = "Amarillo"
        else:
            compliance_status = "Necesita Mejoras"
            color_code = "Rojo"

        evaluation = {
            'empresa_id': company['ID_EMPRESA'],
            'estrellas': stars,
            'evaluacion': {
                'nivel_riesgo': risk_level,
                'estado_cumplimiento': compliance_status,
                'codigo_color': color_code,
                'metricas_clave': {
                    'score_riesgo': round(company['risk_score'], 2),
                    'proporcion_indefinidos': round(company['prop_indefinidos'], 2),
                    'tiempo_operacion': f"{int(company['Tiempo_E_anio'])} años",
                    'indice_seguridad': f"{round(company['ntrab_consctr']/company['numtra']*100, 1)}%"
                }
            },
            'recomendaciones': self.generate_recommendations(company, stars)
        }

        return evaluation

    def generate_recommendations(self, company, stars):
        recommendations = []

        if company['risk_score'] > 7:
            recommendations.append("URGENTE: Implementar sistema de gestión de riesgos más robusto")
        elif company['risk_score'] > 4:
            recommendations.append("Revisar y mejorar las medidas de control de riesgos actuales")

        if company['prop_indefinidos'] < 0.5:
            recommendations.append("Mejorar la estabilidad laboral aumentando contratos indefinidos")

        if company['ntrab_consctr']/company['numtra'] < 0.9:
            recommendations.append("Incrementar cobertura de seguro complementario de trabajo de riesgo")

        if stars <= 3:
            recommendations.append("Desarrollar plan integral de mejora de cumplimiento normativo")

        return recommendations

def main():
    print("Inicializando sistema de evaluación...")
    evaluator = RobustComplianceEvaluator()

    print("Cargando datos...")
    if not evaluator.load_data('/content/data_total.csv'):
        return

    print("Entrenando modelo robusto...")
    if not evaluator.train_model():
        return

    while True:
        print("\n=== Sistema de Evaluación de Cumplimiento Normativo ===")
        company_id = input("\nIngrese ID de empresa (o 'q' para salir): ")

        if company_id.lower() == 'q':
            break

        result = evaluator.evaluate_company(company_id)

        if 'error' in result:
            print(f"\nError: {result['error']}")
            print("IDs válidos disponibles (primeros 10):")
            for id in result['valid_ids'][:10]:
                print(f"- {id}")
            continue

        print("\n=== Resultado de la Evaluación ===")
        print(f"Empresa ID: {result['empresa_id']}")
        print(f"Calificación: {'⭐' * result['estrellas']} ({result['estrellas']} estrellas)")
        print(f"Estado: {result['evaluacion']['estado_cumplimiento']} ({result['evaluacion']['codigo_color']})")

        print("\nEvaluación Detallada:")
        print(f"- Nivel de Riesgo: {result['evaluacion']['nivel_riesgo']}")

        print("\nMétricas Clave:")
        for metric, value in result['evaluacion']['metricas_clave'].items():
            print(f"- {metric}: {value}")

        print("\nRecomendaciones:")
        for i, rec in enumerate(result['recomendaciones'], 1):
            print(f"{i}. {rec}")

if __name__ == "__main__":
    main()

Inicializando sistema de evaluación...
Cargando datos...
Datos cargados exitosamente. Shape: (104, 48)
Columnas disponibles: 48
Entrenando modelo robusto...

Distribución de estrellas:
1    14
2     2
3    20
4    18
5    50
Name: count, dtype: int64

Datos de entrenamiento: (83, 14)
Datos de prueba: (21, 14)

Preparando datos para LightGBM...
Entrenando modelo robusto...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[19]	valid_0's multi_logloss: 0.674414	valid_0's multi_error: 0

Precisión del modelo: 100.00%

Matriz de confusión:
[[ 3  0  0  0]
 [ 0  4  0  0]
 [ 0  0  4  0]
 [ 0  0  0 10]]

Características más importantes:
             feature  importance
1        MONTO_MULTA         110
0         risk_score          23
11        risk_ratio          11
9   prop_indefinidos           3
2     ntrab_tc_indet           2

=== Sistema de Evaluación de Cumplimiento Normativo ===

Ingrese ID de empresa (o 'q' para salir): ZAYVYCXVEWE

=== Re

KeyboardInterrupt: Interrupted by user

#Orquestacion de Agentes AI:
#- Agente AI Predictivo con la data Sunafil - Sunat
#- Agente AI RAG para las Normativas UE
#- Agente AI RAG para recomendaciones

In [ ]:
import os
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import warnings
import lightgbm as lgb
from sk-learn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

warnings.filterwarnings('ignore')

class DataProcessor:
    def __init__(self):
        self.label_encoders = {}
        self.scaler = StandardScaler()

    def preprocess_company_data(self, df):
        df_processed = df.copy()

        # Procesar características numéricas
        numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
        for feature in numeric_features:
            if feature in df_processed.columns:
                df_processed[feature] = df_processed[feature].fillna(df_processed[feature].median())

        # Procesar características categóricas
        categorical_features = df.select_dtypes(include=['object']).columns
        for feature in categorical_features:
            if feature in df_processed.columns:
                if feature not in self.label_encoders:
                    self.label_encoders[feature] = LabelEncoder()
                df_processed[feature] = self.label_encoders[feature].fit_transform(
                    df_processed[feature].fillna('MISSING')
                )

        # Características derivadas
        if all(col in df_processed.columns for col in ['MONTO_MULTA', 'numtra', 'costosal', 'ntrab_consctr']):
            df_processed['risk_ratio'] = df_processed['MONTO_MULTA'] / df_processed['numtra'].replace(0, 1)
            df_processed['worker_density'] = df_processed['numtra'] / df_processed['costosal'].replace(0, 1)
            df_processed['safety_index'] = df_processed['ntrab_consctr'] / df_processed['numtra'].replace(0, 1)

        # Asegurar que exista la columna risk_score si no existe
        if 'risk_score' not in df_processed.columns:
            df_processed['risk_score'] = df_processed['risk_ratio'] * 10

        return df_processed

    def preprocess_single_company(self, company_data):
        """
        Procesa los datos de una única empresa
        """
        if isinstance(company_data, pd.Series):
            company_df = pd.DataFrame([company_data])
        else:
            company_df = pd.DataFrame([company_data])

        return self.preprocess_company_data(company_df)

class RobustLightGBM:
    def __init__(self):
        self.model = None
        self.feature_names = None

    def train(self, X_train, y_train, X_test, y_test):
        self.feature_names = X_train.columns.tolist()

        print("\nPreparando datos para LightGBM...")
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

        params = {
            'objective': 'multiclass',
            'num_class': 5,
            'metric': ['multi_logloss', 'multi_error'],
            'num_leaves': 31,
            'learning_rate': 0.03,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'min_child_samples': 20,
            'verbose': -1
        }

        print("Entrenando modelo")
        callbacks = [
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]

        self.model = lgb.train(
            params,
            train_data,
            num_boost_round=1000,
            valid_sets=[valid_data],
            callbacks=callbacks
        )

        y_pred = self.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred - 1)  # Ajustamos por el offset
        print(f"\nPrecisión del modelo: {accuracy:.2%}")

    def predict(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame([X])
        probabilities = self.model.predict(X[self.feature_names])
        return np.argmax(probabilities, axis=1) + 1

    def predict_single(self, X):
        predictions = self.predict(X)
        return predictions[0]

class EUComplianceRAG:
    def __init__(self, regulations_path: str):
        self.regulations_path = regulations_path
        try:
            self.qa_chain = self._initialize_rag()
        except FileNotFoundError:
            print(f"Advertencia: No se encontró el archivo de regulaciones en {regulations_path}")
            self.qa_chain = None

    def _initialize_rag(self) -> RetrievalQA:
        loader = TextLoader(self.regulations_path)
        documents = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        splits = text_splitter.split_documents(documents)

        embeddings = OpenAIEmbeddings()
        vectorstore = FAISS.from_documents(splits, embeddings)

        llm = ChatOpenAI(
            temperature=0,
            model="gpt-4o-mini"
        )

        qa_template = """Eres un experto en normativas de la Unión Europea.
        Analiza el siguiente contexto y determina si cumple con las normativas:

        Contexto: {context}

        Pregunta: {question}

        Respuesta detallada:"""

        PROMPT = PromptTemplate(
            template=qa_template,
            input_variables=["context", "question"]
        )

        chain_type_kwargs = {"prompt": PROMPT}

        return RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(),
            chain_type_kwargs=chain_type_kwargs,
            return_source_documents=True
        )

    def analyze_compliance(self, company_data: Dict) -> Dict:
        if self.qa_chain is None:
            return {
                "analysis": "No se pudo realizar el análisis normativo por falta de acceso al archivo de regulaciones.",
                "sources": []
            }

        query = f"""
        Analiza si la siguiente empresa cumple con las normativas de la UE:
        - Número de trabajadores: {company_data.get('numtra', 'No disponible')}
        - Proporción de contratos indefinidos: {company_data.get('prop_indefinidos', 'No disponible')}
        - Índice de seguridad: {company_data.get('safety_index', 'No disponible')}
        - Score de riesgo: {company_data.get('risk_score', 'No disponible')}

        Proporciona un análisis detallado del cumplimiento normativo.
        """

        try:
            result = self.qa_chain({"query": query})
            return {
                "analysis": result["result"],
                "sources": [doc.page_content for doc in result["source_documents"]]
            }
        except Exception as e:
            return {
                "analysis": f"Error en el análisis: {str(e)}",
                "sources": []
            }

class RecommendationAgent:
    def __init__(self):
        self.llm = ChatOpenAI(
            temperature=0.7,
            model="gpt-4-turbo-preview"
        )

    def generate_recommendations(self,
                               company_data: Dict,
                               compliance_analysis: Dict,
                               star_rating: int) -> List[str]:
        try:
            prompt = f"""
            Como experto en cumplimiento normativo de la UE, genera recomendaciones
            específicas para mejorar el cumplimiento de la siguiente empresa:

            Datos de la empresa:
            - Calificación actual: {star_rating} estrellas
            - Número de trabajadores: {company_data.get('numtra', 'No disponible')}
            - Índice de seguridad: {company_data.get('safety_index', 'No disponible')}
            - Score de riesgo: {company_data.get('risk_score', 'No disponible')}

            Análisis de cumplimiento previo:
            {compliance_analysis.get('analysis', 'No disponible')}

            Genera 3-5 recomendaciones específicas y accionables para mejorar el
            cumplimiento normativo.
            """

            response = self.llm.predict(prompt)
            recommendations = [rec.strip() for rec in response.split('\n') if rec.strip()]
            return recommendations[:5]  # Limitamos a 5 recomendaciones

        except Exception as e:
            return [f"Error generando recomendaciones: {str(e)}"]

class IntegratedEUComplianceEvaluator:
    def __init__(self, regulations_path: str, company_data_path: str):
        self.data_processor = DataProcessor()
        self.robust_model = RobustLightGBM()
        self.rag_agent = EUComplianceRAG(regulations_path)
        self.recommendation_agent = RecommendationAgent()

        self.load_and_prepare_data(company_data_path)

    def load_and_prepare_data(self, data_path: str):
        try:
            print("Cargando y preparando datos...")
            self.company_data = pd.read_csv(data_path)
            self.processed_data = self.data_processor.preprocess_company_data(self.company_data)
            self.train_robust_model()
        except FileNotFoundError:
            print(f"Error: No se encontró el archivo de datos en {data_path}")
            raise

    def train_robust_model(self):
        print("\nEntrenando modelo robusto de clasificación...")
        features = self.get_important_features()
        X = self.processed_data[features]

        # Crear compliance_score si no existe
        if 'compliance_score' not in self.processed_data.columns:
            self.processed_data['compliance_score'] = 100 - (self.processed_data['risk_score'] * 10)

        compliance_scores = self.processed_data['compliance_score'].fillna(
            self.processed_data['compliance_score'].mean()
        )

        conditions = [
            (compliance_scores <= 65),
            (compliance_scores > 65) & (compliance_scores <= 75),
            (compliance_scores > 75) & (compliance_scores <= 85),
            (compliance_scores > 85) & (compliance_scores <= 95),
            (compliance_scores > 95)
        ]
        values = [0, 1, 2, 3, 4]
        y = pd.Series(np.select(conditions, values, default=2))

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        self.robust_model.train(X_train, y_train, X_test, y_test)

    def get_important_features(self) -> List[str]:
        base_features = [
            'risk_score',
            'MONTO_MULTA',
            'numtra',
            'costosal',
            'ntrab_consctr',
            'risk_ratio',
            'worker_density',
            'safety_index'
        ]

        # Filtrar solo las características que existen en el DataFrame
        return [f for f in base_features if f in self.processed_data.columns]

    def evaluate_company(self, company_id: str) -> Dict:
        try:
            # Verificar si el ID existe
            if company_id not in self.company_data['ID_EMPRESA'].values:
                return {
                    'error': 'ID de empresa no encontrado',
                    'valid_ids': self.company_data['ID_EMPRESA'].unique().tolist()[:10]
                }

            # Obtener datos de la empresa
            company = self.company_data[
                self.company_data['ID_EMPRESA'] == company_id
            ].iloc[0]

            # Procesar datos
            processed_company = self.data_processor.preprocess_single_company(company)
            features = self.get_important_features()

            # Predicción
            star_rating = self.robust_model.predict_single(processed_company[features])

            # Análisis y recomendaciones
            compliance_analysis = self.rag_agent.analyze_compliance(processed_company.to_dict())
            recommendations = self.recommendation_agent.generate_recommendations(
                processed_company.to_dict(),
                compliance_analysis,
                star_rating
            )

            return self.prepare_detailed_evaluation(
                company,
                star_rating,
                compliance_analysis,
                recommendations
            )

        except Exception as e:
            print(f"Error detallado en la evaluación: {str(e)}")
            return {
                'error': 'Error en la evaluación',
                'valid_ids': self.company_data['ID_EMPRESA'].unique().tolist()[:10]
            }

    def prepare_detailed_evaluation(self,
                                  company: pd.Series,
                                  star_rating: int,
                                  compliance_analysis: Dict,
                                  recommendations: List[str]) -> Dict:

        # Calcular métricas si existen los datos necesarios
        risk_score = company.get('risk_score', 0)
        risk_level = "Alto" if risk_score > 7 else "Medio" if risk_score > 4 else "Bajo"

        if star_rating >= 4:
            compliance_status = "Excelente"
            color_code = "Verde"
        elif star_rating == 3:
            compliance_status = "Bueno"
            color_code = "Amarillo"
        else:
            compliance_status = "Necesita Mejoras"
            color_code = "Rojo"

        evaluation = {
            'empresa_id': company['ID_EMPRESA'],
            'estrellas': star_rating,
            'evaluacion': {
                'nivel_riesgo': risk_level,
                'estado_cumplimiento': compliance_status,
                'codigo_color': color_code,
                'analisis_normativo': compliance_analysis['analysis'],
                'metricas_clave': {
                    'score_riesgo': round(risk_score, 2),
                    'proporcion_indefinidos': round(company.get('prop_indefinidos', 0), 2),
                    'tiempo_operacion': f"{int(company.get('Tiempo_E_anio', 0))} años",
                    'indice_seguridad': f"{round(company.get('ntrab_consctr', 0)/company.get('numtra', 1)*100, 1)}%" if 'ntrab_consctr' in company and 'numtra' in company else 'No disponible'
                }
            },
            'recomendaciones': recommendations,
            'fuentes_normativas': compliance_analysis.get('sources', [])
        }

        return evaluation

def check_and_create_dummy_files():
    """
    Crea archivos dummy si no existen los originales para testing
    """
    # Crear archivo dummy de normativas si no existe
    if not os.path.exists('/content/normas_ue.txt'):
        print("Creando archivo dummy de normativas...")
        dummy_regulations = """
        Normativa 1: Las empresas deben mantener un índice de seguridad superior al 80%.
        Normativa 2: El ratio de trabajadores indefinidos debe ser superior al 50%.
        Normativa 3: Las empresas deben implementar medidas de seguridad laboral.
        """
        with open('/content/normas_ue.txt', 'w', encoding='utf-8') as f:
            f.write(dummy_regulations)

    # Crear archivo dummy de datos si no existe
    if not os.path.exists('/content/data_total.csv'):
        print("Creando archivo dummy de datos...")
        dummy_data = pd.DataFrame({
            'ID_EMPRESA': ['EMP001', 'EMP002', 'EMP003'],
            'numtra': [100, 150, 200],
            'MONTO_MULTA': [1000, 2000, 1500],
            'ntrab_consctr': [80, 120, 180],
            'costosal': [50000, 75000, 100000],
            'risk_score': [3, 6, 8],
            'prop_indefinidos': [0.7, 0.6, 0.5],
            'Tiempo_E_anio': [5, 8, 12]
        })
        dummy_data.to_csv('/content/data_total.csv', index=False)

def main():
    # Configurar API key de OpenAI
    os.environ['OPENAI_API_KEY'] = 'sk'

    try:
        # Verificar y crear archivos dummy si es necesario
        check_and_create_dummy_files()

        print("Inicializando sistema integrado de evaluación...")
        evaluator = IntegratedEUComplianceEvaluator(
            regulations_path='/content/normas_ue.txt',
            company_data_path='/content/data_total.csv'
        )

        while True:
            print("\n=== Sistema Integrado de Evaluación de Cumplimiento UE ===")
            print("Opciones disponibles:")
            print("1. Evaluar empresa")
            print("2. Ver IDs de empresas disponibles")
            print("q. Salir")

            option = input("\nSeleccione una opción: ")

            if option.lower() == 'q':
                break

            elif option == '1':
                company_id = input("\nIngrese ID de empresa: ")
                result = evaluator.evaluate_company(company_id)

                if 'error' in result:
                    print(f"\nError: {result['error']}")
                    if 'valid_ids' in result:
                        print("\nIDs válidos disponibles (primeros 10):")
                        for id in result['valid_ids']:
                            print(f"- {id}")
                    continue

                # Mostrar resultados
                print("\n=== Resultado de la Evaluación ===")
                print(f"Empresa ID: {result['empresa_id']}")
                print(f"Calificación: {'⭐' * result['estrellas']} ({result['estrellas']} estrellas)")
                print(f"Estado: {result['evaluacion']['estado_cumplimiento']} ({result['evaluacion']['codigo_color']})")
                print(f"Nivel de Riesgo: {result['evaluacion']['nivel_riesgo']}")

                print("\nMétricas Clave:")
                for metric, value in result['evaluacion']['metricas_clave'].items():
                    print(f"- {metric}: {value}")

                print("\nAnálisis de Cumplimiento Normativo UE:")
                print(result['evaluacion']['analisis_normativo'])

                print("\nRecomendaciones:")
                for i, rec in enumerate(result['recomendaciones'], 1):
                    print(f"{i}. {rec}")

                if result['fuentes_normativas']:
                    print("\nFuentes Normativas Relevantes:")
                    for i, source in enumerate(result['fuentes_normativas'], 1):
                        print(f"\nFuente {i}:")
                        print(source[:200] + "..." if len(source) > 200 else source)

            elif option == '2':
                print("\nIDs de empresas disponibles (primeros 10):")
                valid_ids = evaluator.company_data['ID_EMPRESA'].unique().tolist()[:10]
                for id in valid_ids:
                    print(f"- {id}")

            else:
                print("\nOpción no válida. Por favor, seleccione una opción válida.")

    except Exception as e:
        print(f"\nError crítico: {str(e)}")
        print("El programa se cerrará.")
        raise

if __name__ == "__main__":
    main()

Inicializando sistema integrado de evaluación...
Cargando y preparando datos...

Entrenando modelo robusto de clasificación...

Preparando datos para LightGBM...
Entrenando modelo robusto...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[18]	valid_0's multi_logloss: 0.684955	valid_0's multi_error: 0

Precisión del modelo: 100.00%

=== Sistema Integrado de Evaluación de Cumplimiento UE ===
Opciones disponibles:
1. Evaluar empresa
2. Ver IDs de empresas disponibles
q. Salir

Seleccione una opción: ZAYVYDBBZVY

Opción no válida. Por favor, seleccione una opción válida.

=== Sistema Integrado de Evaluación de Cumplimiento UE ===
Opciones disponibles:
1. Evaluar empresa
2. Ver IDs de empresas disponibles
q. Salir

Seleccione una opción: 1

Ingrese ID de empresa: ZAYVYDBBZVY

=== Resultado de la Evaluación ===
Empresa ID: ZAYVYDBBZVY
Calificación: ⭐⭐⭐⭐⭐ (5 estrellas)
Estado: Excelente (Verde)
Nivel de Riesgo: Bajo

Métricas Clave:
- score_rie